In [1]:
import pandas as pd
import numpy as np
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
from cobra.io import read_sbml_model

pd.set_option('display.max_rows', 500)

In [2]:
from cobra.flux_analysis import flux_variability_analysis

## Abbreviations

 - CCM: Central Carbon Metabolism
 - GSM: Genome-Scale Model

### Worflow

1. Set up model with the central carbon metabolism reactions of interest, the CCM model
2. Import iJN1462 P. putida model, the GSM model
3. Set the fluxes of the reactions in the GSM model that are also part of the CCM model to the fluxes of the CCM model
4. Set the fluxes of the exchange reactions
5. Run pFBA
6. Export the pFBA results

In [3]:
def build_ccm_model(reactions):
    """ Build Central Carbon Metabolism model for the given reactions"""
    
    ccm_model = Model('putida_no_gth')
    excluded_rxns = ['R_ACCOAr', 'R_AXPr', 'R_NADHr', 'R_NADPHr', 'R_Q8H2r', 'R_OPNAD', 'R_OPQ8']
    orig_ccm_rxns_ids = []
    
    for i in range(reactions['rxn_id'].shape[0]):
        r = Reaction(reactions['rxn_id'][i])
        ccm_model.add_reaction(r);
        r.build_reaction_from_string(reactions['rxn_stoic'][i]);
        ccm_model.reactions.get_by_id(reactions['rxn_id'][i]).lower_bound = reactions['lb'][i]
        ccm_model.reactions.get_by_id(reactions['rxn_id'][i]).upper_bound = reactions['ub'][i]

        if not reactions['rxn_id'][i].startswith('R_EX_') and reactions['rxn_id'][i] not in excluded_rxns:
            orig_ccm_rxns_ids.append(reactions['rxn_id'][i])
            
    return ccm_model, orig_ccm_rxns_ids


In [4]:
def get_ids(object_list):
    """ Get all ids from a cobrapy objects, e.g. reactions or metabolites"""
    return [obj.id for obj in object_list]

def find_complete_id(rxn_id, rxn_id_set):
    """ Find a given id given only a partial match"""
    
    complete_rxn_ids = [complete_rxn_id for complete_rxn_id in rxn_id_set if complete_rxn_id.find(rxn_id) !=-1]
    return complete_rxn_ids


In [5]:
ccm_reactions = pd.read_excel('../../models/cobra/PputidaCCM_reactions_scope.xlsx')
ccm_model, orig_ccm_rxns_ids = build_ccm_model(ccm_reactions)


--------------------------------------------
--------------------------------------------

Using license file /Users/svevol/gurobi.lic
Academic license - for non-commercial use only
unknown metabolite 'm_glc__D_p' created
unknown metabolite 'm_atp_c' created
unknown metabolite 'm_h2o_c' created
unknown metabolite 'm_glc__D_c' created
unknown metabolite 'm_adp_c' created
unknown metabolite 'm_pi_c' created
unknown metabolite 'm_g6p_c' created
unknown metabolite 'glc__D_e' created
unknown metabolite 'glc__D_p' created
unknown metabolite 'm_glcn_p' created
unknown metabolite 'm_glcn_c' created
unknown metabolite 'm_6pgc_c' created
unknown metabolite 'm_2dhglcn_p' created
unknown metabolite 'm_2dhglcn_c' created
unknown metabolite 'm_6p2dhglcn_c' created
unknown metabolite 'm_nadph_c' created
unknown metabolite 'm_nadp_c' created
unknown metabolite 'm_q8_c' created
unknown metabolite 'm_q8h2_c' created
unknown metabolite 'm_6pgl_c' created
unknown metabolite 'm_co2_c' created
unknown meta

In [6]:
ccm_rxns_ids=['GLCabcpp',
 'HEX1',
 'GLCtex',          
 'GLCNt2rpp',  'GNK',  '2DHGLCNkt_tpp', '2DHGLCK', 'PGLCNDH', 'GLCDpp','GAD2ktpp', 'G6PDH2r', 'PGL',
 'GND', 'RPI', 'RPE', 'TKT1', 'TKT2', 'TALA', 'EDD', 'EDA', 'PGI', 'FBP',
 'FBA', 'TPI', 'GAPD', 'PGK', 'PGM', 'ENO', 'PYK', 'PDH', 'OAADC', 'PPC', 'ME2', 'MDH', 'FUM', 'SUCDi',
 'SUCOAS', 'AKGDH', 'ICDHyr', 'ACONTa', 'ACONTb', 'CS', 'ICL', 'MALS']

In [7]:
gsm_model = read_sbml_model('../../models/cobra/iJN1463.xml')


## Define fluxes of GSM model based on the CCM model

In [8]:
# define GSM fluxed based on CCM model

for rxn_id in ccm_rxns_ids:
    print(rxn_id)
    if rxn_id in ['PGL', 'PDH']:
        
        gsm_model.reactions.get_by_id(rxn_id).lower_bound = ccm_model.reactions.get_by_id(f'R_{rxn_id}').lower_bound
        gsm_model.reactions.get_by_id(rxn_id).upper_bound = ccm_model.reactions.get_by_id(f'R_{rxn_id}').upper_bound
        
        print(ccm_model.reactions.get_by_id(f'R_{rxn_id}'))
        print(f'[{gsm_model.reactions.get_by_id(rxn_id).lower_bound}, {gsm_model.reactions.get_by_id(rxn_id).upper_bound}]')
        print(f'[{ccm_model.reactions.get_by_id(f"R_{rxn_id}").lower_bound}, {ccm_model.reactions.get_by_id(f"R_{rxn_id}").upper_bound}]')
    else:
        ccm_rxn_id = find_complete_id(rxn_id, orig_ccm_rxns_ids)[0]
        gsm_model.reactions.get_by_id(rxn_id).lower_bound = ccm_model.reactions.get_by_id(ccm_rxn_id).lower_bound
        gsm_model.reactions.get_by_id(rxn_id).upper_bound = ccm_model.reactions.get_by_id(ccm_rxn_id).upper_bound
    
    print(ccm_model.reactions.get_by_id(ccm_rxn_id))
    print(gsm_model.reactions.get_by_id(rxn_id))
    print(f'[{gsm_model.reactions.get_by_id(rxn_id).lower_bound}, {gsm_model.reactions.get_by_id(rxn_id).upper_bound}]')
    print(f'[{ccm_model.reactions.get_by_id(ccm_rxn_id).lower_bound}, {ccm_model.reactions.get_by_id(ccm_rxn_id).upper_bound}]')
    print('')
    


GLCabcpp
R_GLCabcpp: m_atp_c + m_glc__D_p + m_h2o_c --> m_adp_c + m_glc__D_c + m_pi_c
GLCabcpp: atp_c + glc__D_p + h2o_c --> adp_c + glc__D_c + h_c + pi_c
[0.59, 0.67]
[0.59, 0.67]

HEX1
R_HEX1: m_atp_c + m_glc__D_c --> m_adp_c + m_g6p_c
HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
[0.59, 0.67]
[0.59, 0.67]

GLCtex
R_GLCtex: glc__D_e --> glc__D_p
GLCtex: glc__D_e --> glc__D_p
[6.14, 6.14]
[6.14, 6.14]

GLCNt2rpp
R_GLCNt2rpp: m_glcn_p --> m_glcn_c
GLCNt2rpp: glcn_p + h_p --> glcn_c + h_c
[4.72, 4.86]
[4.72, 4.86]

GNK
R_GNK: m_atp_c + m_glcn_c --> m_6pgc_c + m_adp_c
GNK: atp_c + glcn_c --> 6pgc_c + adp_c + h_c
[4.72, 4.86]
[4.72, 4.86]

2DHGLCNkt_tpp
R_2DHGLCNkt_tpp: m_2dhglcn_p --> m_2dhglcn_c
2DHGLCNkt_tpp: 2dhglcn_p + h_p --> 2dhglcn_c + h_c
[0.66, 0.78]
[0.66, 0.78]

2DHGLCK
R_2DHGLCK: m_2dhglcn_c + m_atp_c --> m_6p2dhglcn_c + m_adp_c
2DHGLCK: 2dhglcn_c + atp_c + 2.0 h_c --> 6p2dhglcn_c + adp_c
[0.66, 0.78]
[0.66, 0.78]

PGLCNDH
R_PGLCNDH: m_6p2dhglcn_c + m_nadph_c --> m_6pgc_c + 

In [9]:
# make sure that the CCM model and the GSM model include the same reactions
gsm_rxns_ids = get_ids(gsm_model.reactions)
print(len(set(gsm_rxns_ids).intersection(set(ccm_rxns_ids))))
print(len(ccm_rxns_ids))
set(ccm_rxns_ids).difference(set(gsm_rxns_ids))

44
44


set()

In [10]:
# set correct flux direction for glucose exchange reaction
gsm_model.reactions.get_by_id('EX_glc__D_e').lower_bound = -ccm_model.reactions.get_by_id('R_GLCtex').upper_bound
gsm_model.reactions.get_by_id('EX_glc__D_e').upper_bound = -ccm_model.reactions.get_by_id('R_GLCtex').lower_bound 

In [11]:
# fix the flux for exchange reactions for glcn and 2dhglcn
gsm_model.reactions.get_by_id('EX_glcn_e').lower_bound = gsm_model.reactions.get_by_id('EX_glcn_e').upper_bound = 0
gsm_model.reactions.get_by_id('EX_2dhglcn_e').lower_bound = gsm_model.reactions.get_by_id('EX_2dhglcn_e').upper_bound = 0

### Run pFBA

In [12]:
pfba_solution = pfba(gsm_model)

In [13]:
pfba_solution.fluxes.BIOMASS_KT2440_WT3

0.33778749192786256

In [14]:
pfba_solution.fluxes.EX_glc__D_e

-6.14

In [15]:
#Add constrain that TKT1/15=TKT2
tkt1_tkt2 = gsm_model.problem.Constraint(
    15*gsm_model.reactions.TKT2.flux_expression - gsm_model.reactions.TKT1.flux_expression,
    lb=0,
    ub=0)
gsm_model.add_cons_vars(tkt1_tkt2)

In [16]:
pfba_solution = pfba(gsm_model)

In [17]:
pfba_solution.fluxes.BIOMASS_KT2440_WT3

0.32477208426787824

In [18]:
pfba_solution.fluxes.EX_glc__D_e

-6.14

### Get final fluxes and convert from mmol/gCDW/h to mmol/L/h

mmol/L/h = mmol/gCDW/h * gCDW / cell_volume

In [19]:
# values from Ishii
gCDW = 2.8*10**-13
cell_volume = 0.74*(4.96*10**-16)

In [20]:
pfba_solution.fluxes[ccm_rxns_ids+['CYTBO3_4pp', 'NADH16pp', 'ATPS4rpp','ACCOAC', 'MCOATA', 'KAS15']]

GLCabcpp          0.620000
HEX1              0.620000
GLCtex            6.140000
GLCNt2rpp         4.860000
GNK               4.860000
2DHGLCNkt_tpp     0.660000
2DHGLCK           0.660000
PGLCNDH           0.660000
GLCDpp            5.520000
GAD2ktpp          0.660000
G6PDH2r           1.170000
PGL               1.170000
GND               0.620000
RPI              -0.456809
RPE               0.157108
TKT1              0.147289
TKT2              0.009819
TALA              0.141641
EDD               6.070000
EDA               6.070000
PGI              -0.595288
FBP               0.469529
FBA              -0.469529
TPI              -0.490000
GAPD              5.070000
PGK              -5.150000
PGM              -4.430000
ENO               4.430000
PYK               3.970000
PDH               6.490000
OAADC            -4.590000
PPC              -0.020000
ME2               2.510000
MDH               1.900000
FUM               4.846019
SUCDi             4.495689
SUCOAS           -4.490000
A

In [31]:
pfba_solution.fluxes[ccm_rxns_ids+['CYTBO3_4pp', 'NADH16pp', 'ATPS4rpp','ACCOAC', 'MCOATA', 'KAS15']] * gCDW/cell_volume

GLCabcpp           472.972973
HEX1               472.972973
GLCtex            4683.958152
GLCNt2rpp         3707.497820
GNK               3707.497820
2DHGLCNkt_tpp      503.487358
2DHGLCK            503.487358
PGLCNDH            503.487358
GLCDpp            4210.985179
GAD2ktpp           503.487358
G6PDH2r            892.545772
PGL                892.545772
GND                472.972973
RPI               -348.480853
RPE                119.851659
TKT1               112.360930
TKT2                 7.490729
TALA               108.051966
EDD               4630.557977
EDA               4630.557977
PGI               -454.121360
FBP                358.185046
FBA               -358.185046
TPI               -373.801221
GAPD              3867.698344
PGK              -3928.727114
PGM              -3379.468178
ENO               3379.468178
PYK               3028.552746
PDH               4950.959024
OAADC            -3501.525719
PPC                -15.257193
ME2               1914.777681
MDH       

### Export pfba fluxes

In [33]:
round(pfba_solution.fluxes[ccm_rxns_ids+['CYTBO3_4pp', 'NADH16pp', 'ATPS4rpp','ACCOAC', 'MCOATA', 'KAS15']] * gCDW/cell_volume, 0).to_csv('data/pfba_fluxes_with_tkt2.csv')